In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
files = os.listdir('./clouds')
files = ['./clouds/' + i for i in files]

In [3]:
from scipy.io import loadmat

In [4]:
def cloud_to_dual_maps(cloud):
    
    dual = np.zeros((450, 450, 2))
    
#     img = plt.imread(m['img'][0])[66:,50:434]/255.
    
    cloud = cloud.T
    cloud = cloud[cloud[:,2].argsort()]

    xs = np.round(cloud[:,0]).astype('i')
    ys = np.round(cloud[:,1]).astype('i')
    zs = cloud[:,2] - np.min(cloud[:,2])
    z_max = np.max(zs)
    zs1 = z_max - zs
    
    cloud_dict = {}
    for x, y, z in zip(xs, ys, zs):
        if (x, y) in cloud_dict: cloud_dict[(x, y)] = max(z, cloud_dict[(x, y)])
        else: cloud_dict[(x, y)] = z
    
    for i in cloud_dict:
        t = dual[447 - i[1]:452 - i[1], i[0]-2:i[0]+3, 1]
        t[t<cloud_dict[i]] = cloud_dict[i]
        dual[447 - i[1]:452 - i[1], i[0]-2:i[0]+3, 1] = t

    cloud_dict2 = {}
    for x, y, z in zip(xs[::-1], ys[::-1], zs1[::-1]):
        if (x, y) in cloud_dict2: cloud_dict2[(x, y)] = max(z, cloud_dict2[(x, y)])
        cloud_dict2[(x, y)] = z   
    
    for i in cloud_dict2:
        t = dual[447 - i[1]:452 - i[1], i[0]-2:i[0]+3, 0]
        t[t<cloud_dict2[i]] = cloud_dict2[i]
        dual[447 - i[1]:452 - i[1], i[0]-2:i[0]+3, 0] = t
    
    dual = dual[66:,50:434,:].astype('uint8')
    
    return dual

In [5]:
print('memory required :', len(files)*289/1024/1024, 'GB')

memory required : 16.874337196350098 GB


In [6]:
for i in tqdm(range(0, 60000, 1000)):
    images = []
    for j, fname in tqdm(enumerate(files[i:i+1000])):
        m = loadmat(fname)
        cloud = m['ProjectVertex']
        try:
            local_name = 'dual_maps/dual_' + str(i + j) + '.npy'
            images.append([m['img'][0], local_name])
            np.save(local_name, cloud_to_dual_maps(cloud))
        except: continue
    np.save('img_maps/img_' + str(i) + '.npy', images)

In [ ]:
m = loadmat(fname)
cloud = m['ProjectVertex']
c = cloud_to_dual_maps(cloud)